# Initial Data Exploration and data processing

### Pulling Dataset from [TrashNet](https://github.com/garythung/trashnet)

In [2]:
# import shutil

import requests, zipfile, io
dataset_url = "https://github.com/garythung/trashnet/raw/master/data/dataset-resized.zip"

r = requests.get(dataset_url, stream=True)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()


### Building npz package for FEDn

In [3]:
import shutil
import random
import os
import glob
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split

dir_path = "dataset-resized"

# Pointing dataset to path
if os.path.exists("__MACOSX"):
    shutil.rmtree("__MACOSX")
if os.path.exists(f"{dir_path}/.DS_Store"):
    os.remove(f"{dir_path}/.DS_Store")

dir_labels = os.listdir(dir_path)

labeled_images = []
for label in dir_labels:
    labeled_images.append(glob.glob(os.path.join(dir_path, f'{label}/*')))
print(f"Found {len(dir_labels)} types of data images.\n")

image_shape = (300, 300)

img_data, labels = [], []
for index, label in enumerate(dir_labels):
    print(f"Processing {label} images...", end="")
    for img_path in labeled_images[index]:
        image = Image.open(img_path).resize(image_shape)
        img_data.append(np.asarray(image))
        labels.append(index)
    print(" Done.")
print(f"\nCollected {len(img_data)} images of {len(dir_labels)} types")

Found 6 types of data images.

Processing cardboard images... Done.
Processing glass images... Done.
Processing metal images... Done.
Processing paper images... Done.
Processing plastic images... Done.
Processing trash images... Done.

Collected 2527 images of 6 types


In [4]:
print("Building data package")
x_train, x_test, y_train, y_test = train_test_split(img_data, labels, test_size=0.20, random_state=42)

np.savez_compressed('trashNet', x_train=x_train, x_test=x_test, y_train=y_train, y_test=y_test)
print("data package saved as trashnet.npz")

Building data package
data package saved as trashnet.npz


In [5]:
## cleaning up images
# shutil.rmtree(dir_path)